In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats

# for regressions with statsmodels:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.compat import lzip
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.graphics.regressionplots import plot_leverage_resid2

# for regressions with scikit-learn:
import sklearn.linear_model as sklm
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report, precision_score, \
                            accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score

#for plotting confusion matrix:
import scikitplot as skplt

#for ordinal logistic regression
from mord import LogisticIT

#for KNN
# for KNN:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor

In [37]:
df = pd.read_excel("processed_CIR.xlsx")

df.reindex()

,Unnamed: 0,Segment,Category,Region,A,B,C,D,E,F,G,H,I,J,K,L,AVG,Year
0,2,INTERNET,Retail Price ($ per pair),NorthA,67.00,67.000,67.00,67.00,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,67.0,Y10
1,3,INTERNET,Search Engine Advert. ($000s),NorthA,5000.00,5000.000,5000.00,5000.00,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,Y10
2,4,INTERNET,Free Shipping,NorthA,0.00,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y10
3,6,INTERNET,S/Q Rating (1 to 10 stars),NorthA,4.00,4.000,4.00,4.00,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Y10
4,7,INTERNET,Model Availability,NorthA,200.00,200.000,200.00,200.00,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,Y10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,43,PRIVATELABEL,Offer Price (max = $40.00),LA,43.69,39.000,59.98,39.49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67.5,Y18
1076,44,PRIVATELABEL,S/Q Rating (min = 3.0 stars),LA,5.50,5.000,5.00,5.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,Y18
1077,46,PRIVATELABEL,Pairs Offered (000s),LA,383.00,995.000,296.00,1003.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2008.0,Y18
1078,47,PRIVATELABEL,Pairs Sold (000s),LA,0.00,995.000,0.00,1003.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2677.0,Y18


In [38]:
A_team_df = df[["Segment", "Category", "Region", "A", "Year"]]
A_team_df
table = pd.pivot_table(A_team_df, values='A', columns="Category", index = ["Year", "Region", "Segment"])
table

,Segment,Category,Region,A,Year
0,INTERNET,Retail Price ($ per pair),NorthA,67.00,Y10
1,INTERNET,Search Engine Advert. ($000s),NorthA,5000.00,Y10
2,INTERNET,Free Shipping,NorthA,0.00,Y10
3,INTERNET,S/Q Rating (1 to 10 stars),NorthA,4.00,Y10
4,INTERNET,Model Availability,NorthA,200.00,Y10
...,...,...,...,...,...
1075,PRIVATELABEL,Offer Price (max = $40.00),LA,43.69,Y18
1076,PRIVATELABEL,S/Q Rating (min = 3.0 stars),LA,5.50,Y18
1077,PRIVATELABEL,Pairs Offered (000s),LA,383.00,Y18
1078,PRIVATELABEL,Pairs Sold (000s),LA,0.00,Y18


Category                     Gained / Lost (due to stockouts)  \
Year Region Segment                                             
Y10  AP     INTERNET                                      NaN   
            PRIVATELABEL                                  NaN   
            WHOLESALE                                     0.0   
     EP     INTERNET                                      NaN   
            PRIVATELABEL                                  NaN   
...                                                       ...   
Y18  LA     PRIVATELABEL                                  NaN   
            WHOLESALE                                     0.0   
     NorthA INTERNET                                      NaN   
            PRIVATELABEL                                  NaN   
            WHOLESALE                                    27.0   

Category                   Brand Advertising ($000s)  \
Year Region Segment                                    
Y10  AP     INTERNET                          8000.0   
            PRIVATELABEL                         NaN   
            WHOLESALE                         8000.0   
     EP     INTERNET                          9000.0   
            PRIVATELABEL                         NaN   
...                                              ...   
Y18  LA     PRIVATELABEL                         NaN   
            WHOLESALE                        30000.0   
     NorthA INTERNET                         30000.0   
            PRIVATELABEL                         NaN   
            WHOLESALE                        30000.0   

Category                   Brand Reputation (prior-year image)  \
Year Region Segment                                              
Y10  AP     INTERNET                                      70.0   
            PRIVATELABEL                                   NaN   
            WHOLESALE                                     70.0   
     EP     INTERNET                                      70.0   
            PRIVATELABEL                                   NaN   
...                                                        ...   
Y18  LA     PRIVATELABEL                                   NaN   
            WHOLESALE                                     86.0   
     NorthA INTERNET                                      86.0   
            PRIVATELABEL                                   NaN   
            WHOLESALE                                     86.0   

Category                   Celebrity Appeal   Delivery Time (weeks)  \
Year Region Segment                                                   
Y10  AP     INTERNET                    0.0                     NaN   
            PRIVATELABEL                NaN                     NaN   
            WHOLESALE                   0.0                     3.0   
     EP     INTERNET                    0.0                     NaN   
            PRIVATELABEL                NaN                     NaN   
...                                     ...                     ...   
Y18  LA     PRIVATELABEL                NaN                     NaN   
            WHOLESALE                 135.0                     1.0   
     NorthA INTERNET                  145.0                     NaN   
            PRIVATELABEL                NaN                     NaN   
            WHOLESALE                 145.0                     1.0   

Category                   Free Shipping   Market Share (%)  \
Year Region Segment                                           
Y10  AP     INTERNET                 0.0             0.2500   
            PRIVATELABEL             NaN             0.2500   
            WHOLESALE                NaN             0.2500   
     EP     INTERNET                 0.0             0.2500   
            PRIVATELABEL             NaN             0.2500   
...                                  ...                ...   
Y18  LA     PRIVATELABEL             NaN             0.0000   
            WHOLESALE                NaN             0.2413   
     NorthA 

In [33]:
# figure out EPS 
# get costs for year 

Cost = table[" Brand Advertising ($000s)"]["Y10"] * 1000 + table[" Search Engine Advert. ($000s)"] * 1000
Income = table[" Pairs Sold (000s)"]["Y10"] * 1000 * table[" Retail Price ($ per pair)"]

net = Income - Cost
net

Year
Y10     8651000.0
Y11    10225000.0
Y12    12402000.0
Y13    16300000.0
Y14    15873500.0
Y15    15932500.0
Y16    15742500.0
Y17    13242500.0
Y18    13242500.0
dtype: float64

In [27]:
fir_df = pd.read_excel("mikes_processed_CIR.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'mikes_processed_CIR.xlsx'